<a href="https://colab.research.google.com/github/Doongka/GHDColabExamples/blob/master/ExcelPlot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Upload Excel Spreadsheet and Plot Example
The goal of this notebook is to provide a quick overview of some useful Python tools used for cleaning, analysing and visualizing data.


## Prepare the workspace
Google Colab has many useful Python packages preinstalled such so there shouldn't be any need to install them yourself.  Packages are basically collections of useful functions.  If case you do need something that is not installed, you can run a "pip" installation as per below.  

You can see that I've "commented" the bit of code that installs the package "".  Uncomment this line and run the code block.


In [0]:
# We place a ! in front of the command to indicate that we want this to run in a console.
# !pip install altair --upgrade

## Uploading your data to Colab
After running the cell below, you will see a "Choose Files" button that will allow you to select the files you wish to upload.  




In [19]:
from google.colab import files

uploaded = files.upload()

Saving Main_Meter_flow_logger_data.xlsx to Main_Meter_flow_logger_data (1).xlsx


Running the next cell will give you an overview of the files you have just uploaded.  This is a handy test to make sure it uploaded correctly.

In [20]:
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

User uploaded file "Main_Meter_flow_logger_data.xlsx" with length 696921 bytes


## Wrangling your data

Python has many pre-built packages that help with any extract, transform and load (ETL) operations required to get the data ready.  The standard library contains lots of really good methods but sometimes we want something even better.

Pandas is a popular library that provides many useful methods for data manipulation. We will be using the "Dataframe" functionality which will allow us to import data from a CSV file and perform several data wrangling and cleansing tasks.



In [0]:
#These commands allow us to load the libraries in to our current notebook
import pandas as pd

## Loading your data
Before we can do anything with our data, we first need to load it.  Pandas has some great methods to read files and load them into a useful format. 

In the code block below, we are loading a CSV file and tranforming it in the form of a Dataframe object.  A Dataframe is effectively Pandas's answer to storing data in a tabular format.

In [22]:
# read the csv and load the contents into memory as a dataframe
# data = pd.read_csv('brisbanetemp.csv')

# read the excel and load the contents into memory as a dataframe
data = pd.read_excel('Main_Meter_flow_logger_data.xlsx')

# the head() method allows us to view the first 5 entries in the dataframe
data.head()

,Graph Unit,Kilolitres
0,NaN,NaN
1,NaN,Botanical Gardens Toowong Mobile Logger #107 (...
2,Wednesday 20/03/2019 12:00:00 AM,0
3,Wednesday 20/03/2019 12:15:00 AM,0.03
4,Wednesday 20/03/2019 12:30:00 AM,0.04


# Cleaning up your data
From the table above check to see if any header rows are unable to be read.  "NaN" means "Not a Number" and in this case was caused by empty cells in the first row of the data.  Let's remove this row entry using the following command. 

In [23]:
# Drop the rows from the dataframe and assign to a new Dataframe called "cleansed_data"
cleansed_data = data.drop(data.index[0:2])

# Let's have a look to see if it worked
cleansed_data.head()

,Graph Unit,Kilolitres
2,Wednesday 20/03/2019 12:00:00 AM,0
3,Wednesday 20/03/2019 12:15:00 AM,0.03
4,Wednesday 20/03/2019 12:30:00 AM,0.04
5,Wednesday 20/03/2019 12:45:00 AM,0
6,Wednesday 20/03/2019 01:00:00 AM,0.08


It's also worth checking that the data types are correct

In [24]:
cleansed_data.dtypes

Graph Unit     object
 Kilolitres    object
dtype: object

If you need to convert to a number type like float or int you can do the following

In [0]:
cleansed_data[' Kilolitres'] = pd.to_numeric(cleansed_data[' Kilolitres'])

If there are columns not needed, you can drop these too.

In [0]:
# drop the columns 'XXXXXXX'.  Replace XXXXXXX with the appropriate column header
# cleansed_data = cleansed_data.drop(['XXXXXXX'], axis=1)

# check to see what we're left with
# cleansed_data.head()

Looking good but we should check the rest of the data.  Running the "info" method gives us information on the type of data stored in each column and also the number of non-null (not empty or NaN) entries. 

In [27]:
cleansed_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36211 entries, 2 to 36212
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Graph Unit   36211 non-null  object 
 1    Kilolitres  36211 non-null  float64
dtypes: float64(1), object(1)
memory usage: 848.7+ KB


If the number of non-null entries for each column is not equal, this means there are null entries sitting in our table.  We can search for them by doing the following.

In [0]:
# Find the null values in the column named XXXXXX
# missing_data = cleansed_data['XXXXXX)'].isna()

# find the null values in another column YYYYYY.  We are using a logical OR (|) to get a combined list of null entries
# missing_data = missing_data | cleansed_data['YYYYYY)'].isna()

# Count the number of rows that have null entries
# missing_data.sum()

If rows contain at least one null value look at them using this:-

In [0]:
# Show only the rows that contain the null values.  Note the format.  The input number represents the rows we want, the second is the columns we want.  A colon (:)
# means we want to show all columns
# cleansed_data.loc[missing_data,:]

We have several options at out disposal here to handle the null entries, we can:

  

*   delete the rows but that's generally not a good idea for time series data 
*   back or forward filling the entry using the values around it
*   interpolate between values

Uncomment the approach you'd like to take


In [30]:
# We can remove all of the rows that have NaNs.  Probably not a good idea for a time series plot but necessary if the data can't be imputed
# final_data = cleansed_data.dropna(how='any')

# We can back fill the data based on the data either side
# final_data = cleansed_data.fillna(method='bfill')
# Or forward fill
# final_data = cleansed_data.fillna(method='ffill')

# We can interpolate 
# final_data = cleansed_data.interpolate(method='linear')

# If there are no changes use this
final_data = cleansed_data
final_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36211 entries, 2 to 36212
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Graph Unit   36211 non-null  object 
 1    Kilolitres  36211 non-null  float64
dtypes: float64(1), object(1)
memory usage: 848.7+ KB


The last thing we need to do is to make sure the date is in a usable format for the graph.  We can do this by using the "to_datetime" method in Pandas

In [31]:
# Fix the data column so it is in a usable format
final_data['Graph Unit'] = pd.to_datetime(final_data['Graph Unit'],infer_datetime_format=True)

# Rename the columns so they're easier to read.  Chnage XXXXXX to YYYYYY
# final_data = final_data.rename(columns={"XXXXXX": "YYYYYY"})
final_data = final_data.rename(columns={"Graph Unit": "Date", " Kilolitres": "Kilolitres"})

# take a final look at the data
final_data.head()

,Date,Kilolitres
2,2019-03-20 00:00:00,0.00
3,2019-03-20 00:15:00,0.03
4,2019-03-20 00:30:00,0.04
5,2019-03-20 00:45:00,0.00
6,2019-03-20 01:00:00,0.08


We can also get a summary of key statistics by using the .describe() command


In [32]:
# Get a the summay stats from the dataset
display(final_data.astype(int).describe())
display(final_data.describe())


,Date,Kilolitres
count,3.621100e+04,36211.000000
mean,1.569767e+18,2.389164
std,1.302302e+16,5.138577
min,1.546560e+18,0.000000
25%,1.559632e+18,0.000000
50%,1.568557e+18,0.000000
75%,1.577482e+18,2.000000
max,1.607126e+18,60.000000


,Kilolitres
count,36211.000000
mean,2.726612
std,5.212633
min,0.000000
25%,0.050000
50%,0.670000
75%,2.910000
max,60.780000


## Visualizing your Data


Altair is a great data visualization library that is preinstalled in Colab.  It is a statistical visualization language with lots of plot types and statistical functions.



In [37]:
# load up the Altair package to give us access to advanced visualization methods
import altair as alt

# Altair has a default 5000 entry limit, this can be turned off but may be slow
alt.data_transformers.disable_max_rows()

#X axis range
X_min = '2019-01-04 00:00:00'
X_max =  '2020-1-30 23:45:00'

#Y axis range
volume_limit_max = 60
volume_limit_min = 0

# We need to create a mask to pull out the data we want.  This basically tells us the row indexes that sit between the two dates
mask = (final_data['Date'] > X_min) & (final_data['Date'] <= X_max)

# We'll create a new dataframe to store our plot data.  The .loc[mask] method returns the entries based on the indexes we obtained above
plot_data = final_data.loc[mask]

#Check number of data points
plot_data.describe()


,Kilolitres
count,27763.000000
mean,3.069781
std,5.503602
min,0.000000
25%,0.140000
50%,1.010000
75%,3.450000
max,60.780000


Altair is a bit involved to produce a plot and takes a while to get used to but it produces beautiful plots.  The example below is an interactive scatterplot that allows us to zoom in and out.

In [38]:
# The code below produces an interactive scatterplot that lets us zoom in and out
alt.Chart(plot_data, height=500, width=1000).mark_circle().encode(
    alt.X('Date:T',
          scale=alt.Scale(zero=False)
    ),
    alt.Y('Kilolitres:Q',
          scale=alt.Scale(zero=False)
    ),
    tooltip=['Date', 'Kilolitres'],
    color=alt.Color('Kilolitres', sort='descending', scale=alt.Scale(scheme=alt.SchemeParams(name='redyellowblue')))
).interactive()


alt.Chart(...)